In [1]:
import sys

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from sklearn import metrics
from sklearn.model_selection import train_test_split

sys.path.append('../src')
import columnar as col

%load_ext autoreload
%autoreload 2

In [3]:
df = col.loaders.petfinder._load('../')
feature_selection = col.FeatureSelection(**col.loaders.petfinder._select_features(df))

# split data
df_train, df_test = train_test_split(df, test_size=.2)

X_train, y_train = feature_selection.select_features(df_train)
ds_train = col.embeddings.data.df_to_dataset(X_train, y_train, batch_size=32)

X_test, y_test = feature_selection.select_features(df_test)
ds_test = col.embeddings.data.df_to_dataset(X_test, y_test, batch_size=32, shuffle=False)

# setup scorer
scorer = col.Scorer(
    acc=lambda ytrue, ypreds: metrics.accuracy_score(ytrue, ypreds > .5),
    f1=lambda ytrue, ypreds: metrics.f1_score(ytrue, ypreds > .5),
    auc=metrics.roc_auc_score,
)

Metal device set to: Apple M1 Pro


2022-02-16 11:05:46.450881: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-16 11:05:46.451022: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# testing config

In [15]:
cfg = col.config.BenchmarkConfig.load('../config.yml')
cfg.paths

Paths(root='./', figures='figures/', reports='reports/')

In [16]:
cfg

BenchmarkConfig(paths=Paths(root='./', figures='figures/', reports='reports/'), metrics=MetricsConfig(include_acc=True, include_f1=True, include_auc=True), classifiers=ClassifiersConfig(rf__n_estimators=100, rf__max_depth=10, lr__max_iter=500, knn__n_neighbors=10), transformers=TransformersConfig(include_mte=True, include_tfembeddings_single=True, include_tfembeddings_max2=True, include_tfembeddings_max50=True, include_onehot=True, include_ordinal=True, mte__alpha=5, tfclassifier__hidden_size='int = 32', tfclassifier__dropout_rate='float = .5'), cross_validation=CrossValidationParams(n_splits=5), fit_params=FitParams(tf__epochs=3, tf__verbose=1), plot=PlotConfig(style='fivethirtyeight'))